In [2]:
import tensorflow as tf
from keras.layers import Embedding, Dense, LSTM
from keras.losses import BinaryCrossentropy
from keras.models import Sequential
from keras.optimizers import Adam

In [3]:
import numpy as np

def generate_general_mip(num_vars, num_constrs, dens, rng, max_constr_coeff=100, max_obj_coeff=100, max_solution_value=100, ensure_solution=True, logging=False):
    """Generate general MIP instance.
    TODO: fill in generation details.
    :param num_vars: number of variables
    type: int, should be positive
    :param num_constrs: number of constraints
    type: int, should be positive
    :param dens: density of constraint matrix
    type: float, should be in (0, 1)
    :param rng: random number generator
    type: `np.random.RandomState` object or other objects that have `randint` method
    :param max_constr_coeff: maximal value of constraints coefficient. By default 100.
    type: int, should be positive
    :param max_obj_coeff: maximal value of objective coefficient. By default 100.
    type: int, should be positive
    :param max_solution_value: maximal value of solution coefficient. By default 100.
    type: int, should be positive
    :param ensure_solution: whether to ensure a integral solution in the problem. By default True
    type: bool
    :param logging: whether to print the logging info
    type: bool

    :returns: (A, b, c, integral_list)
        A, b, c: parameter for MIP problem, in standard format `max c @ x,  s.t. A @ x <= b`.
        integral_list: whether the variable is integer. 1 means the variable at the corresponding
            position is integral.
        sense: sense of the objective, "MIN" or "MAX".
    :rtype:
        A: np.array of shape (num_constrs, num_vars)
        b: np.array of shape (num_vars,)
        c: np.array of shape (num_vars,)
        integral_list: np.array of shape (num_vars,)
        sense: string
    """
    nnzrs = int(num_constrs * num_vars * dens)
    assert nnzrs >= num_constrs and nnzrs >= 2 * num_vars

    row_nz_elem_split = sorted(rng.choice(np.arange(1, nnzrs), num_constrs, replace=False).tolist(), reverse=True) + [0]
    row_nz_elem_num = np.array([row_nz_elem_split[i] - row_nz_elem_split[i+1] for i in range(num_constrs)])
    col_idx_list = rng.randint(low=0, high=num_vars, size=(nnzrs,))
    col_idx_list[rng.choice(nnzrs, num_vars * 2, replace=False)] = np.repeat(np.arange(num_vars), 2)

    if logging:
        print("Number of non-zero elements for each row:\n\t", end="")
        print(row_nz_elem_num)
        print("Column index of non-zero elements:\n\t", end="")
        print(col_idx_list)

    ensured_solution = rng.randint(low=0, high=max_solution_value, size=(num_vars,))
    if logging:
        print("Ensured solution:\n\t", ensured_solution)

    A_list = []
    b_list = []
    elem_idx = 0
    for i in range(num_constrs):
        a = np.zeros(shape=(num_vars,))
        a[col_idx_list[elem_idx : elem_idx + row_nz_elem_num[i]]] = \
            rng.randint(low=1, high=max_constr_coeff, size=(row_nz_elem_num[i],))
        if ensure_solution:
            b = rng.randint(low=a @ ensured_solution.T, high=5 * a @ ensured_solution.T)
        else:
            b = rng.randint(high=max_constr_coeff * max_solution_value)
        A_list.append(a)
        b_list.append(b)
        elem_idx += row_nz_elem_num[i]

    A = np.vstack(A_list)
    b = np.hstack(b_list)
    c = rng.randint(low=1, high=max_obj_coeff, size=num_vars)

    if logging:
        print("A:\n\t", A)
        print("b:\n\t", b)
        print("c:\n\t", c)
        if ensure_solution:
            print("Checking A @ ensured_solution <= b:\n\t", np.all(A @ ensured_solution.T <= b))

    return A, b, c, np.ones(shape=(num_vars,)), "MAX"


In [13]:
A, b, c, integral_list, sense = generate_general_mip(3, 2, 1, np.random.RandomState(0), logging=True)

Number of non-zero elements for each row:
	[2 1]
Column index of non-zero elements:
	[2 1 1 0 2 0]
Ensured solution:
	 [39 87 46]
A:
	 [[ 0. 82. 89.]
 [ 0. 78.  0.]]
b:
	 [28149 13002]
c:
	 [10 21 81]
Checking A @ ensured_solution <= b:
	 True


In [ ]:
# Model configuration
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 15
loss_function = BinaryCrossentropy()
max_sequence_length = 300
num_distinct_words = 5000
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Disable eager execution
tf.compat.v1.disable_eager_execution()

# Load dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_distinct_words)
print(x_train.shape)
print(x_test.shape)

# Pad all sequences
padded_inputs = pad_sequences(x_train, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>
padded_inputs_test = pad_sequences(x_test, maxlen=max_sequence_length, value = 0.0) # 0.0 because it corresponds with <PAD>

# Define the Keras model
model = Sequential()
model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
model.summary()

# Train the model
history = model.fit(padded_inputs, y_train, batch_size=batch_size, epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)

# Test the model after training
test_results = model.evaluate(padded_inputs_test, y_test, verbose=False)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {100*test_results[1]}%')